### Import Libraries

In [1]:
#Pretrained Model EfficientNetB1 without Top Layer
# !wget "https://storage.googleapis.com/keras-applications/efficientnetb1_notop.h5"

In [4]:
import tensorflow as tf
import os
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam

----

### Create model

In [5]:
#Use pretrained model as transfer learning layers
local_weight_file = local_weight_file = './pretrained/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

#Adjust input shape and weights
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

In [6]:
# Load the downloaded pre-trained weights
pre_trained_model.load_weights(local_weight_file)

# Freeze the weights of the layers.
for layer in pre_trained_model.layers:
  layer.trainable = False

In [7]:
#Specify input layer model
last_layer = pre_trained_model.get_layer('mixed5')
print('last layer output shape: ', last_layer.output_shape)

last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [10]:
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.MaxPooling2D()(last_output)
x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
# Add a dropout rate of 0.3
x = layers.Dropout(0.3)(x) 
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)         

# Append the dense network to the base model
acneModel = Model(pre_trained_model.input, x) 

# Print the model summary
acneModel.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                            

In [11]:
#Compiled the model using adam optimizer and binary classification
acneModel.compile(optimizer = Adam(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [12]:
base_dir = '../Dataset/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with training acne pictures
train_acne_dir = os.path.join(train_dir, 'acne3') 

# Directory with training normal pictures
train_normal_dir = os.path.join(train_dir, 'normal2') 

# Directory with validation acne pictures
validation_acne_dir = os.path.join(validation_dir, 'acne2') 

# Directory with validation normal pictures
validation_normal_dir = os.path.join(validation_dir, 'normal2')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=['acne3', 'normal2'],
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         classes=['acne2', 'normal2'],
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

Found 408 images belonging to 2 classes.
Found 159 images belonging to 2 classes.


In [13]:
#Defining callback for preventing underfitting and overfitting
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and (logs.get('accuracy') > 0.99 or logs.get('accuracy') < 0.5):
            print("\nCancelling training")
            self.model.stop_training = True

In [ ]:
#Train the model
callbacks = myCallback()
history = acneModel.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 25,
            callbacks=[callbacks])

----

### Visualizing Accuracy Plot

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation Loss')
plt.legend(loc=0)
plt.figure()
plt.show()

----

### Try Input Image

In [1]:
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tensorflow.keras.utils import load_img, img_to_array

#Code for open dialog window on Local Machine
root = tk.Tk()
root.withdraw()
uploaded = filedialog.askopenfilename()
 
#Predicting images
path = #PathtoImages
img = load_img(path, target_size=(224, 224))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = acneModel.predict(images, batch_size=10)
print(classes[0])

if classes[0]>0.5:
  print("no dullness/acne")
  print()
else:
  print("dullness/acne")
  print()
 

----

### Convert and Save Model

In [18]:
import pickle

dump = {
    'acne':acneModel,
}
# Save your model to a file using pickle
with open("acneModel.pkl", "wb") as f:
    pickle.dump(dump, f)

In [17]:
acneModel.save("acneModel")

INFO:tensorflow:Assets written to: acneModel\assets


INFO:tensorflow:Assets written to: acneModel\assets


In [16]:
acneModel.save("acneModel.h5")